# TimeAttentionレイヤを実装する

In [1]:
import numpy as np

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    print('「ファイルを選択」から、notebook/commonフォルダのfunctions.py, layers.pyを選択し、アップロードしてください')
    print('===========')
    files.upload()
    !mkdir common
    !mv *.py ./common
except:
    print('ローカル環境での実行です')


from common.layers import Softmax

Google Colab. 上での実行です
「ファイルを選択」から、notebook/commonフォルダのfunctions.py, layers.pyを選択し、アップロードしてください


Saving functions.py to functions.py
Saving layers.py to layers.py


### [演習]
* 以下のWeightSum, AttentionWeight, Attention, TimeAttentionクラスを完成させましょう

In [2]:
class WeightSum:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None

    def forward(self, hs, a):
        """
        順伝播
        hs : エンコーダの中間状態
        a : アテンション荷重
        """
        N, T, H = hs.shape

        # アテンション荷重の行列を3次元配列に変形する
        ar = a.reshape(N, T, 1)#.repeat(H, axis=2)   ブロードキャストを明示的に行いたい場合はrepeatを付ける
        # エンコーダの中間状態にアテンション荷重をかけて、それを足し合わせることによって、加重平均を求める
        t = hs * ar
        c = np.sum(t, axis=1)

        self.cache = (hs, ar)
        return c  # エンコーダの中間状態を加重平均した結果

    def backward(self, dc):
        """
        逆伝播
        """
        hs, ar = self.cache
        N, T, H = hs.shape
        dt = dc.reshape(N, 1, H).repeat(T, axis=1)
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2)

        return dhs, da


class AttentionWeight:
    """
    アテンション荷重を算出するクラス
    """
    def __init__(self):
        self.params, self.grads = [], []
        self.softmax = Softmax()
        self.cache = None

    def forward(self, hs, h):
        """
        順伝播
        アテンション荷重を求める
        hs : エンコーダの全ての中間状態
        h : デコーダのある場所の中間状態
        """
        N, T, H = hs.shape

        #　デコーダのある場所の中間状態を3次元配列に変形する
        hr = h.reshape(N, 1, H)#.repeat(T, axis=1)
        
        # エンコーダの中間状態とデコーダの中間状態を掛けて足し合わせることで内積をとる
        # 他の実装例として、hsとhrを結合し、重みWを掛けるという方法もある
        t = hs * hr
        s = np.sum(t, axis=2)
        
        # ソフトマックス関数に通すことで、正規化する
        a = self.softmax.forward(s) # アテンション重みベクトルを並べた行列 (N * T)

        self.cache = (hs, hr)
        return a

    def backward(self, da):
        """
        逆伝播
        """
        hs, hr = self.cache
        N, T, H = hs.shape

        ds = self.softmax.backward(da)
        dt = ds.reshape(N, T, 1).repeat(H, axis=2)
        dhs = dt * hr
        dhr = dt * hs
        dh = np.sum(dhr, axis=1)

        return dhs, dh


class Attention:
    """
    アテンション
    """
    def __init__(self):
        self.params, self.grads = [], []
        
        # レイヤの定義
        self.attention_weight_layer = AttentionWeight()
        self.weight_sum_layer = WeightSum()
        self.attention_weight = None

    def forward(self, hs, h):
        """
        順伝播
        hs : エンコーダの中間状態
        h : デコーダの中間状態
        """
        # アテンション荷重を求める
        a = self.attention_weight_layer.forward(hs, h)
        
        # エンコーダの中間状態にアテンション荷重をかける
        out = self.weight_sum_layer.forward(hs, a)
        self.attention_weight = a
        
        return out # エンコーダの中間状態を加重平均した結果

    def backward(self, dout):
        """
        逆伝播
        """
        dhs0, da = self.weight_sum_layer.backward(dout)
        dhs1, dh = self.attention_weight_layer.backward(da)
        dhs = dhs0 + dhs1
        return dhs, dh


class TimeAttention:
    """
    アテンションレイヤを時間方向にまとめるレイヤ
    """
    def __init__(self):
        self.params, self.grads = [], []
        self.layers = None
        self.attention_weights = None

    def forward(self, hs_enc, hs_dec):
        """
        順伝播
        hs_enc : エンコーダの中間状態
        hs_dec : デンコーダの中間状態
        """
        N, T, H = hs_dec.shape
        out = np.empty_like(hs_dec)
        self.layers = []
        self.attention_weights = []

        for t in range(T):
            """
            出力単語数分を繰り返す
            """
            layer = Attention()
            out[:, t, :] = layer.forward(hs_enc, hs_dec[:,t,:]) 
            self.layers.append(layer)
            self.attention_weights.append(layer.attention_weight)

        return out

    def backward(self, dout):
        """
        逆伝播
        dout : 勾配
        """
        N, T, H = dout.shape
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)

        for t in range(T):
            """
            出力単語数分を繰り返す
            """
            layer = self.layers[t]
            dhs, dh = layer.backward(dout[:, t, :])
            dhs_enc += dhs
            dhs_dec[:,t,:] = dh

        return dhs_enc, dhs_dec


In [3]:
# 中間層ノード数
H = 4
# データ数
N = 3
# 単語数
T = 5


# モデル構築
ta = TimeAttention()

hs_enc = np.random.randn(N*T*H).reshape(N, T, H)
hs_dec =  np.random.randn(N*T*H).reshape(N, T, H)
print("hs_enc=", hs_enc)
print()
print("hs_dec=", hs_dec)
print()

# 順伝播計算
out = ta.forward(hs_enc, hs_dec)
print("out=", out)
print()

# 逆伝播計算
dout = np.random.randn(N*T*H).reshape(N, T, H)
dhs_enc, dhs_dec = ta.backward(dout)
print("dhs_enc=", dhs_enc)
print()
print("dhs_dec=", dhs_dec)
print()

hs_enc= [[[ 0.8364571  -0.28069335 -0.8320408  -0.50688231]
  [ 0.73796366  0.49924237 -0.82906364  0.86554529]
  [-0.73735882 -0.18003811  0.12667723  0.12328764]
  [ 0.51966551 -0.23062502 -1.55888864  0.72147356]
  [ 0.8200234  -0.18223576 -0.25593208  0.15045162]]

 [[-2.1807734   0.84315985  0.4789938  -0.42880736]
  [-0.39409279 -0.44445982 -1.80595027 -0.73794791]
  [-0.55825832  0.24170041  0.51492901  0.86062029]
  [-0.38414615  1.21795234 -0.04734203  0.39272002]
  [-1.03825381  0.09060522  0.60678377  0.17138226]]

 [[ 0.56314598 -0.97664923  0.17844121  0.99951956]
  [ 1.71451251  1.09905897  0.80675279  0.40584776]
  [-1.53113429 -0.26343648 -0.6634367   0.03206651]
  [-0.22972906  0.85766222 -1.64693639  0.62993046]
  [ 1.93996987 -0.33427568 -0.95317805  0.50026314]]]

hs_dec= [[[ 0.27541914  0.61709845 -2.89106771 -0.53897574]
  [-0.77490151  0.49161161 -0.37140959 -2.19366288]
  [-0.45744571 -0.82710011  0.07379446  0.14558411]
  [ 0.38992893 -0.99376803 -2.09787485 -1